In [62]:
# load all needed libraries

import pandas as pd
import pickle
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.cluster import KMeans
#from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.metrics import classification_report
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score,explained_variance_score,mean_absolute_error
#from sklearn.model_selection import GridSearchCV

folder = 'C:/Users/szums/AIBootCampPrime/AgProject3/Resources/'

In [71]:
def crop_prediction(measure,factorarray):
    modelmap = {
'TOBACCO_$_ACRE' : ['TOBACCO_$_ACREgbr_model.pkl','H'],
'WHEAT_BU_ACRE' : ['WHEAT_BU_ACREgbr_model.pkl','H'],
'SQUASH_CWT_ACRE' : ['SQUASH_CWT_ACREdtr_model.pkl','H'],
'SOYBEANS_BU_ACRE' : ['SOYBEANS_BU_ACRElr_model.pkl','H'],
'PEANUTS_LB_ACRE' : ['PEANUTS_LB_ACRErfr_model.pkl','H'],
'PEPPERS,BELL_$_ACRE' : ['PEPPERS,BELL_$_ACREsvr_model.pkl','H'],
'SOYBEANS_$_ACRE' : ['SOYBEANS_$_ACREgbr_model.pkl','M'],
'BARLEY_BU_ACRE' : ['BARLEY_BU_ACREdtr_model.pkl','M'],
'CORN_BU_ACRE' : ['CORN_BU_ACREgbr_model.pkl','M'],
'HAY_$_ACRE' : ['HAY_$_ACRErfr_model.pkl','M'],
'BARLEY_$_ACRE' : ['BARLEY_$_ACREgbr_model.pkl','M'],
'OATS_$_ACRE' : ['OATS_$_ACREsvr_model.pkl','M'],
'COTTON_LB_ACRE' : ['COTTON_LB_ACREsvr_model.pkl','M'],
'TOBACCO_LB_ACRE' : ['TOBACCO_LB_ACREsvr_model.pkl','M'],
'HAY_T_ACRE' : ['HAY_T_ACREsvr_model.pkl','M'],
'WHEAT_$_ACRE' : ['WHEAT_$_ACREsvr_model.pkl','M'],
'OATS_BU_ACRE' : ['OATS_BU_ACREsvr_model.pkl','M'],
'SQUASH_$_ACRE' : ['SQUASH_$_ACREsvr_model.pkl','L'],
'PEANUTS_$_ACRE' : ['PEANUTS_$_ACREsvr_model.pkl','L'],
'CORN_$_ACRE' : ['CORN_$_ACREsvr_model.pkl','L'],
'PEPPERS, BELL_CWT_ACRE' : ['PEPPERS, BELL_CWT_ACREsvr_model.pkl','L'],
'SWEET_$_ACRE' : ['SWEET_$_ACREsvr_model.pkl','L'],
'SWEET_CWT_ACRE' : ['SWEET_CWT_ACREdtr_model.pkl','L']

    }


    #print('entering crop_prediction with',measure,'and',factorarray)

    if measure not in modelmap.keys():
        print('invalid measure',measure,'. Valid measures are\n',modelmap.keys())
        return None
    
    modelfile = folder + modelmap[measure][0]
    #print(modelfile)
    scalar_X_file = modelfile.replace("model","X")
    scalar_y_file = modelfile.replace("model","y")
    #print('about to open files',modelfile,scalar_X_file,scalar_y_file)

    with open(modelfile, 'rb') as file:
        #print('opened',modelfile)
        loaded_model = pickle.load(file)

    with open(scalar_X_file, 'rb') as file:
        #print('opened',scalar_X_file)
        loaded_X_scaler = pickle.load(file)

    with open(scalar_y_file, 'rb') as file:
        #print('opened',scalar_y_file)
        loaded_y_scaler = pickle.load(file)

    #print('loaded',modelfile,scalar_X_file,scalar_y_file)
    
    factorarray = np.array(factorarray).reshape(1, -1)
    #display(factorarray.shape)

    # When making predictions:
    X_scaled = loaded_X_scaler.transform(factorarray)  # Scale new input data
    y_pred_scaled = loaded_model.predict(X_scaled)
    y_pred = loaded_y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1))  # Inverse transform predictions
    
    conf = modelmap[measure][1]

    #print('returning with',y_pred,'and',conf)

    return y_pred[0][0], conf

    
    


In [72]:
#test the crop_prediction function

# factor array should look like this:
# Index(['Fall (SON)', 'Spring (MAM)', 'Summer (JJA)', 'Winter (DJF)',
#        'Precip Fall', 'Precip Spring', 'Precip Summer', 'Precip Winter',
#        'Weeks of Severe drought', 'Weeks of Extreme Drought',
#        'Weeks of Exceptional Drought'],
#       dtype='object')

# get factors
factors1 = [58.77,59.8,75.13,42.6,9.58,11.03,14.9,8.83,29,15,1]
factors2 = [61.2,60.53,76.1,46.7,9.53,16.09,14.34,8.34,17,12,0]
factors3 = [62,59.77,77.3,39.8,19.58,10.4,13.58,11.21,10,0,0]
factors4 = [60.03,57.77,75.2,40.27,5.35,10.2,15.48,6.7,47,29,0]
factors5 = [58.6,58.17,76.4,44.6,10.85,11.79,13.38,9.99,53,52,36]

expected1 = 146.575
expected2 = 380
expected3 = 277.7601156
expected4 = 137.28
expected5 = 307.8898204

# compare results to training/testing time

#prediction1 = crop_prediction('BANANAS PER BUNCH',factors1)
prediction1,conf1 = crop_prediction('SOYBEANS_$_ACRE',factors1)
prediction2,conf2 = crop_prediction('TOBACCO_$_ACRE',factors2)
prediction3,conf3 = crop_prediction('SQUASH_$_ACRE',factors3)
prediction4,conf4 = crop_prediction('SWEET_$_ACRE',factors4)
prediction5,conf5 = crop_prediction('PEANUTS_$_ACRE',factors5)

print('SOYBEANS',prediction1,expected1,conf1)
print('TOBACCO',prediction2,expected2,conf2)
print('SQUASH',prediction3,expected3,conf3)
print('SWEET POTATOES',prediction4,expected4,conf4)
print('PEANUTS',prediction5,expected5,conf5)



SOYBEANS 191.41618464339535 146.575 M
TOBACCO 4274.186196388673 380 H
SQUASH 2835.455465160018 277.7601156 L
SWEET POTATOES 2850.320275258597 137.28 L
PEANUTS 841.5257136347656 307.8898204 L


c:\Users\szums\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\szums\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\szums\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\szums\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\szums\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
